# Harris Corner Detection

In [1]:
from skimage import data, io, filters
import scipy as sp
import numpy as np
import cv2

## Gaussian Smooth

In [2]:
# image = data.camera()
image = io.imread('original.jpg')
io.imshow(image)
io.show()

<Figure size 640x480 with 1 Axes>

In [3]:
def gaussian_kernel(window_size, sigma):
    N = (window_size + 1) // 2

    rhalf_kernel = np.zeros((N,))
    
    for i in range(N):
        x_n = i / sigma
        rhalf_kernel[i] = np.exp(-0.5 * x_n * x_n)

    if window_size % 2 == 0:
        lhalf_kernel = np.flip(rhalf_kernel)
    else:
        lhalf_kernel = np.flip(rhalf_kernel[1:])
        
    kernel = np.concatenate([lhalf_kernel, rhalf_kernel])
    
    kernel /= sum(kernel)
    
    return kernel

In [4]:
def gaussian_smooth(img, k, sigma):
    kernel = gaussian_kernel(k, sigma)
    
    image = img

    for j in range(3):
        layer = image[:,:,j]

        m = layer.shape[1]
        for i in range(m):
            layer[i,:] = np.convolve(layer[i,:], kernel, mode='same')

        n = image.shape[0]
        for i in range(n):
            layer[:,i] = np.convolve(layer[:,i], kernel, mode='same')
        
        image[:,:,j] = layer
    
    return image

## Sobel Edge Detection